# Multi-outputs Task

In [ ]:
!pip install polars[pandas]

In [1]:
import numpy
import pandas # or use `polars`
import torch

## Data Processing from scikit-learn

In [2]:
from sklearn.datasets import make_multilabel_classification

In [3]:
X, y = make_multilabel_classification(n_samples=1000, 
                                      n_features=10,
                                      n_classes=3,
                                      n_labels=2,
                                      random_state=1)

In [4]:
X.shape, y.shape

((1000, 10), (1000, 3))

## Machine Learning Process

### Load Perming and Config Hyperparameters


In [5]:
import perming
main = perming.Box(10, 3, (30,), batch_size=8, activation='relu', inplace_on=True, solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
# main = perming.Ranker(10, 3, (30,), batch_size=16, activation='relu', solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
# main = perming.COMMON_MODELS['Multi-outputs'](10, 3, (30,), batch_size=16, activation='relu', solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
main.print_config()

MLP(
  (mlp): Sequential(
    (Linear0): Linear(in_features=10, out_features=30, bias=True)
    (Activation0): ReLU(inplace=True)
    (Linear1): Linear(in_features=30, out_features=3, bias=True)
  )
)


OrderedDict([('torch -v', '1.7.1+cu101'),
             ('criterion', MultiLabelSoftMarginLoss()),
             ('batch_size', 8),
             ('solver',
              SGD (
              Parameter Group 0
                  dampening: 0
                  lr: 0.01
                  momentum: 0
                  nesterov: False
                  weight_decay: 0
              )),
             ('lr_scheduler', None),
             ('device', device(type='cuda'))])

### Dataloader from Numpy with Multi-threaded


In [6]:
main.data_loader(X, y, random_seed=0)

### Training Stage and Accelerated Validation


In [7]:
main.train_val(num_epochs=60, interval=25, tolerance=1e-4, patience=10, early_stop=True)

Epoch [1/60], Step [25/100], Training Loss: 0.5728, Validation Loss: 0.6097
Epoch [1/60], Step [50/100], Training Loss: 0.5332, Validation Loss: 0.5659
Epoch [1/60], Step [75/100], Training Loss: 0.6924, Validation Loss: 0.5343
Epoch [1/60], Step [100/100], Training Loss: 0.4715, Validation Loss: 0.5089
Epoch [2/60], Step [25/100], Training Loss: 0.3703, Validation Loss: 0.4871
Epoch [2/60], Step [50/100], Training Loss: 0.4212, Validation Loss: 0.4736
Epoch [2/60], Step [75/100], Training Loss: 0.4324, Validation Loss: 0.4670
Epoch [2/60], Step [100/100], Training Loss: 0.4909, Validation Loss: 0.4555
Epoch [3/60], Step [25/100], Training Loss: 0.4825, Validation Loss: 0.4467
Epoch [3/60], Step [50/100], Training Loss: 0.3726, Validation Loss: 0.4297
Epoch [3/60], Step [75/100], Training Loss: 0.3812, Validation Loss: 0.4221
Epoch [3/60], Step [100/100], Training Loss: 0.2042, Validation Loss: 0.4202
Epoch [4/60], Step [25/100], Training Loss: 0.1721, Validation Loss: 0.4101
Epoch [4/

### Test Model with Loss


In [8]:
main.test()

loss of Box on the 104 test dataset: 0.17087210714817047.


OrderedDict([('problem', 'multi-outputs'),
             ('loss',
              {'train': 0.2868916988372803,
               'val': 0.2697790265083313,
               'test': 0.17087210714817047})])

In [9]:
X, y = torch.as_tensor(X, dtype=torch.float).to(torch.device("cuda")), torch.as_tensor(y, dtype=torch.float).to(torch.device("cuda"))

In [10]:
pred = main.model(X) # predicted
# refer to https://pytorch.org/torcheval/main/ for metrics functional tools, like classification
# take input as pred, target as y

In [11]:
'{}%'.format(100 * sum(row.all().int().item() for row in (pred.ge(0.5) == y)) / X.shape[0])

'76.5%'

### Save Model Parameters to Models Folder


In [12]:
main.save(con=False, dir='../models/outputs.ckpt')

### Load Model Parameters from Models Folder

In [13]:
main.load(con=False, dir='../models/outputs.ckpt')